In [1]:
from op_analytics.datasources.github.dataaccess import Github


prs_table = Github.PRS.read()
print(prs_table)

2025-01-27 14:43:09 [info     ] Reading data from 'github/github_prs_v1' with filters min_date=None, max_date=None, date_range_spec=None filename=dailydata.py lineno=187 process=47377
2025-01-27 14:43:09 [info     ] initialized duckdb at /var/folders/tz/g2vwn0qx5ll32j7jfsshhps40000gn/T/u39fvcue/op-analytics.duck.db filename=client.py lineno=94 process=47377
2025-01-27 14:43:09 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=47377
2025-01-27 14:43:09 [debug    ] loaded vault: 18 items         filename=vault.py lineno=79 process=47377
2025-01-27 14:43:14 [info     ] registered view: 'github_github_prs_v1' using uri wildcard 'gs://oplabs-tools-data-sink/github/github_prs_v1/dt=*/out.parquet' filename=client.py lineno=291 process=47377
┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ github_github_prs_v1 │
└──────────────────────┘



In [2]:
from op_analytics.coreutils.duckdb_inmem.client import init_client

ctx = init_client()

ctx.client.sql("SELECT dt, count(*) FROM github_github_prs_v1 GROUP BY 1 ORDER BY 1")

┌────────────┬──────────────┐
│     dt     │ count_star() │
│    date    │    int64     │
├────────────┼──────────────┤
│ 2020-12-31 │          260 │
│ 2021-12-31 │         1316 │
│ 2022-12-31 │         2352 │
│ 2023-12-31 │         3961 │
│ 2025-01-13 │         6411 │
│ 2025-01-15 │          420 │
│ 2025-01-16 │          431 │
│ 2025-01-17 │          443 │
│ 2025-01-18 │          427 │
│ 2025-01-19 │          438 │
│ 2025-01-20 │          436 │
│ 2025-01-21 │          433 │
│ 2025-01-22 │          419 │
│ 2025-01-23 │          408 │
│ 2025-01-24 │          396 │
│ 2025-01-25 │          384 │
│ 2025-01-26 │          390 │
│ 2025-01-27 │          402 │
├────────────┴──────────────┤
│ 18 rows         2 columns │
└───────────────────────────┘

In [6]:
df = ctx.client.sql("SELECT dt, repo, number FROM github_github_prs_v1").pl()
df 

dt,repo,number
date,str,i64
2020-12-31,"""optimism""",1
2020-12-31,"""optimism""",2
2020-12-31,"""optimism""",3
2020-12-31,"""optimism""",4
2020-12-31,"""optimism""",5
…,…,…
2025-01-27,"""supersim""",320
2025-01-27,"""supersim""",322
2025-01-27,"""supersim""",323


In [17]:
import polars as pl

from op_analytics.coreutils.time import date_fromstr
from op_analytics.datasources.github.activity.githubapi import bulk_fetch_reviews, OptimismRepo

# The various original backfill "dt"s
# target_date = "2020-12-31"
target_date = "2021-12-31"


target_dt = date_fromstr(target_date)


dfs = []
for (dt, repo), group_df in df.sort("dt", "repo").group_by("dt", "repo"):
    if dt != target_dt:
        continue
    
    print(dt, repo, len(group_df))    
    repo_obj = OptimismRepo(repo)
    pr_reviews = bulk_fetch_reviews(repo_obj, group_df["number"].to_list())
    
    extracols = dict(repo=pl.lit(repo), dt=pl.lit(target_date))
    dfs.append(pr_reviews.with_columns(**extracols))
    
reviews_df = pl.concat(dfs)


2021-12-31 optimism 1316
2025-01-27 15:24:26 [info     ] fetching reviews for 1316 prs  filename=githubapi.py lineno=99 process=47377
2025-01-27 15:24:29 [info     ] fetching data for prs. completed 1 of 1316 filename=githubapi.py lineno=122 process=47377
2025-01-27 15:24:38 [info     ] fetching data for prs. completed 6 of 1316 filename=githubapi.py lineno=122 process=47377
2025-01-27 15:24:47 [info     ] fetching data for prs. completed 11 of 1316 filename=githubapi.py lineno=122 process=47377
2025-01-27 15:24:58 [info     ] fetching data for prs. completed 16 of 1316 filename=githubapi.py lineno=122 process=47377
2025-01-27 15:25:08 [info     ] fetching data for prs. completed 21 of 1316 filename=githubapi.py lineno=122 process=47377
2025-01-27 15:25:17 [info     ] fetching data for prs. completed 26 of 1316 filename=githubapi.py lineno=122 process=47377
2025-01-27 15:25:27 [info     ] fetching data for prs. completed 31 of 1316 filename=githubapi.py lineno=122 process=47377
2025-01

In [18]:
len(reviews_df)

4014

In [19]:
import os
from unittest.mock import patch

from op_analytics.coreutils.partitioned import dailydata
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.datasources.github.dataaccess import Github

data = dfs

def mock_location():
    return DataLocation.GCS

os.environ["ALLOW_WRITE"] = "true"

with patch.object(dailydata, "determine_location", mock_location):
    Github.PR_REVIEWS.write(
        dataframe=reviews_df,
        sort_by=["repo", "pr_number", "submitted_at"],
    )

2025-01-27 16:17:14 [info     ] done writing 4.0Krows 86.1KB   filename=gcs_parquet.py lineno=57 maxrss=237551616 path=oplabs-tools-data-sink/github/github_pr_reviews_v2/dt=2021-12-31/out.parquet process=47377 root=github/github_pr_reviews_v2 rows=4014 size=86098
2025-01-27 16:17:14 [info     ] memory usage                   filename=gcs_parquet.py lineno=65 max_rss=237.55 process=47377 root=github/github_pr_reviews_v2
2025-01-27 16:17:15 [debug    ] done writing github/github_pr_reviews_v2 to GCS filename=writehelper.py lineno=131 process=47377 root=github/github_pr_reviews_v2


In [20]:
Github.PR_REVIEWS.create_clickhouse_view()

2025-01-27 16:17:43 [info     ] created clickhouse view: github_gcs.github_pr_reviews_v2 filename=gcsview.py lineno=46 process=47377
